In [ ]:
%cd /content
!git clone https://github.com/cg123/ties-merge.git
%cd /content/ties-merge
!pip install -r requirements.txt

In [ ]:
BASE_MODEL = "TheBloke/Llama-2-13B-fp16"
MERGE_MODELS = [
    "WizardLM/WizardLM-13B-V1.2",
    "garage-bAInd/Platypus2-13B",
    # can be model+lora, like so:
    # "TheBloke/CodeLlama-13B-fp16+Blackroot/Llama-2-13B-Storywriter-LORA"
]

# Tweakable parameters
DIFF_DENSITY = 0.33  # fraction of weights from each diff vs. base model to retain
NAIVE_SIGN_COUNT = False  # if True, will use an alternate, dumber sign consensus algorithm

In [ ]:
import torch

# detect device capabilities
use_cuda = False
int8_mask = False
bf16 = False
if torch.cuda.is_available():
  use_cuda = True
  (maj, min) = torch.cuda.get_device_capability()
  if maj > 7 or (maj == 7 and min > 0):
    int8_mask = True
  if torch.cuda.is_bf16_supported():
    bf16 = True

print(f"CUDA: {use_cuda}")
if use_cuda:
  print(f"bf16: {bf16}, int8: {int8_mask}")

In [ ]:
# actually do merge
%cd /content/ties-merge/
import ties_merge

ties_merge.main(
    base_model=BASE_MODEL,
    out_path="/content/merged",
    merge=MERGE_MODELS,
    density=DIFF_DENSITY,
    naive_count=NAIVE_SIGN_COUNT,
    cuda=use_cuda,
    int8_mask=int8_mask,
    bf16=bf16,
    merged_cache_dir="/content/tmp-lora-models/",
)